In [22]:
from keras_tuner import HyperModel
import keras_tuner as kt

import numpy as np
import pandas as pd
#!pip install keras_tuner
#import keras_tuner

import keras
from keras import models, layers, metrics, Sequential
from keras.layers import Dense, BatchNormalization, Dropout
import tensorflow as tf
from keras.regularizers import l1,l2,l1_l2
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Input, Dense, Multiply, Concatenate
from tensorflow.keras.models import Model

import tensorflow as tf
import random

import keras_tuner
from keras_tuner import HyperModel
import keras_tuner as kt

from sklearn.preprocessing import MinMaxScaler

In [23]:
merged_latent = pd.read_pickle("merged_latent49.pkl")

# merged data 1
firm_col = merged_latent.columns.tolist()[18:18+49]
latent_col = merged_latent.columns.tolist()[18+49+1:]
print(len(firm_col), len(latent_col))

49 49


In [24]:
df_train = merged_latent[merged_latent.index < pd.Period((str(1995)+"-1"),freq='M')]
df_val = merged_latent[(merged_latent.index >= pd.Period((str(1995)+"-1"),freq='M')) & (merged_latent.index < pd.Period((str(2000)+"-1"),freq='M'))]
df_test = merged_latent[merged_latent.index >= pd.Period((str(2000)+"-1"),freq='M')]

merged_train_firm = df_train[firm_col]
merged_train_latent = df_train[latent_col]
merged_train_y = df_train['predicted_return']
print(merged_train_firm.shape, merged_train_latent.shape, merged_train_y.shape)

merged_val_firm = df_val[firm_col]
merged_val_latent = df_val[latent_col]
merged_val_y = df_val['predicted_return']
print(merged_val_firm.shape, merged_val_latent.shape, merged_val_y.shape)

merged_test_firm = df_test[firm_col]
merged_test_latent = df_test[latent_col]
merged_test_y = df_test['predicted_return']
print(merged_test_firm.shape, merged_test_latent.shape, merged_test_y.shape)

(88347, 49) (88347, 49) (88347,)
(31851, 49) (31851, 49) (31851,)
(90007, 49) (90007, 49) (90007,)


In [27]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp, n_inputs=49,):
        l1_reg = hp.Choice("l1_ratio", [0.,])
        l2_reg = hp.Choice("l2_ratio", [0.,0.001])
        dropout = hp.Choice("dropout",[ 0., 0.1])
        lr = hp.Choice("learning_rate", [0.0001, 0.00005])
        
        # Define the hidden layers 
        input1 = Input(shape=(49,))
        x1 = Dense(6, activation='relu')(input1)
        
        # Input 2
        input2 = Input(shape=(49,))
        x2 = Dense(32, activation='relu',kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(input2)
        x2 = Dense(16, activation='relu',kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(x2)
        x2 = Dense(8, activation='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(x2)
        x2 = Dense(6, activation='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(x2)
        
        merged = Multiply()([x1, x2])
        output = Dense(1, activation='linear')(merged)
 
        # Create the model 
        model = Model(inputs=[input1, input2], outputs=output)
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [128,256]),
            **kwargs,
        )

In [28]:
max_trials = 200
train_epochs = 200

tuner = keras_tuner.RandomSearch(MyHyperModel(),objective='val_loss',max_trials=max_trials,overwrite=True,)
tuner.search([merged_train_firm, merged_train_latent], merged_train_y, 
             epochs=train_epochs, 
             validation_data=([merged_val_firm, merged_val_latent], merged_val_y),
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

Trial 16 Complete [00h 00m 35s]
val_loss: 0.014558274298906326

Best val_loss So Far: 0.013937660492956638
Total elapsed time: 00h 08m 11s
INFO:tensorflow:Oracle triggered exit


In [29]:
# 10 Trials for Prediction
# 10 Trials for Prediction
merged_train_latent_all = np.concatenate((merged_train_latent, merged_val_latent))
merged_train_firm_all = np.concatenate((merged_train_firm, merged_val_firm))
merged_y_all = np.concatenate((merged_train_y, merged_val_y))
best_hps = tuner.get_best_hyperparameters(5)
print(best_hps[0].values)
print(best_hps[1].values)
print(best_hps[2].values)

{'l1_ratio': 0.0, 'l2_ratio': 0.0, 'dropout': 0.1, 'learning_rate': 0.0001, 'batch_size': 128}
{'l1_ratio': 0.0, 'l2_ratio': 0.0, 'dropout': 0.1, 'learning_rate': 5e-05, 'batch_size': 128}
{'l1_ratio': 0.0, 'l2_ratio': 0.0, 'dropout': 0.1, 'learning_rate': 0.0001, 'batch_size': 256}


In [32]:
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = tuner.get_best_models(3)[2]
    best_model.fit([merged_train_latent, merged_train_firm], merged_train_y,
                   epochs=50,
                   validation_data=([merged_val_latent, merged_val_firm], merged_val_y),
                   callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
    print(best_model.evaluate([merged_test_latent, merged_test_firm], merged_test_y))
    y_hat = best_model.predict([merged_test_latent, merged_test_firm]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

Epoch 1/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0114 - my_metric_fn: 0.0558 - val_loss: 0.0138 - val_my_metric_fn: 0.0092
Epoch 2/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0104 - my_metric_fn: 0.1299 - val_loss: 0.0138 - val_my_metric_fn: 0.0042
Epoch 3/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0102 - my_metric_fn: 0.1421 - val_loss: 0.0137 - val_my_metric_fn: 0.0077
Epoch 4/50
2813/2813 [==============================] - 5s 2ms/step - loss: 0.0132 - my_metric_fn: 0.0541
[0.013186192139983177, 0.05410729721188545]
2813/2813 [==============================] - 4s 1ms/step

Epoch 1/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0113 - my_metric_fn: 0.0587 - val_loss: 0.0139 - val_my_metric_fn: 0.0024
Epoch 2/50
2761/2761 [==============================] - 7s 3ms/step - loss: 0.0104 - my_metric_fn: 0.1290 - val_loss: 0.0138 - val_my_metric_fn: 0.0019
Epoch 3/50
2761/2761 [==========

Epoch 1/50
2761/2761 [==============================] - 7s 3ms/step - loss: 0.0113 - my_metric_fn: 0.0590 - val_loss: 0.0139 - val_my_metric_fn: 7.8615e-04
Epoch 2/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0104 - my_metric_fn: 0.1293 - val_loss: 0.0138 - val_my_metric_fn: 0.0021
Epoch 3/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0102 - my_metric_fn: 0.1431 - val_loss: 0.0138 - val_my_metric_fn: 0.0062
Epoch 4/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0101 - my_metric_fn: 0.1543 - val_loss: 0.0137 - val_my_metric_fn: 0.0057
Epoch 5/50
2813/2813 [==============================] - 5s 2ms/step - loss: 0.0131 - my_metric_fn: 0.0535
[0.01312837190926075, 0.053477901965379715]
2813/2813 [==============================] - 4s 2ms/step

Epoch 1/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0114 - my_metric_fn: 0.0468 - val_loss: 0.0139 - val_my_metric_fn: 0.0070
Epoch 2/50
2761/2761 [======

Epoch 1/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0113 - my_metric_fn: 0.0574 - val_loss: 0.0139 - val_my_metric_fn: 0.0043
Epoch 2/50
2761/2761 [==============================] - 6s 2ms/step - loss: 0.0104 - my_metric_fn: 0.1300 - val_loss: 0.0138 - val_my_metric_fn: 0.0050
Epoch 3/50
2761/2761 [==============================] - 6s 2ms/step - loss: 0.0102 - my_metric_fn: 0.1448 - val_loss: 0.0138 - val_my_metric_fn: 0.0056
Epoch 4/50
2761/2761 [==============================] - 6s 2ms/step - loss: 0.0101 - my_metric_fn: 0.1500 - val_loss: 0.0138 - val_my_metric_fn: 0.0048
Epoch 5/50
2761/2761 [==============================] - 6s 2ms/step - loss: 0.0100 - my_metric_fn: 0.1585 - val_loss: 0.0137 - val_my_metric_fn: 0.0111
Epoch 6/50
2813/2813 [==============================] - 5s 2ms/step - loss: 0.0132 - my_metric_fn: 0.0504
[0.01316971704363823, 0.05035945028066635]
2813/2813 [==============================] - 4s 1ms/step

Epoch 1/50
2761/2761 [===========

Epoch 1/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0114 - my_metric_fn: 0.0574 - val_loss: 0.0139 - val_my_metric_fn: 0.0056
Epoch 2/50
2813/2813 [==============================] - 4s 2ms/step - loss: 0.0133 - my_metric_fn: 0.0624
[0.013254106976091862, 0.06236640363931656]
2813/2813 [==============================] - 4s 1ms/step

0.16671723048501386


In [31]:
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = tuner.get_best_models(3)[1]
    best_model.fit([merged_train_latent, merged_train_firm], merged_train_y,
                   epochs=50,
                   validation_data=([merged_val_latent, merged_val_firm], merged_val_y),
                   callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
    print(best_model.evaluate([merged_test_latent, merged_test_firm], merged_test_y))
    y_hat = best_model.predict([merged_test_latent, merged_test_firm]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

Epoch 1/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0125 - my_metric_fn: -0.0402 - val_loss: 0.0144 - val_my_metric_fn: 0.0097
Epoch 2/50
2761/2761 [==============================] - 7s 3ms/step - loss: 0.0118 - my_metric_fn: 0.0269 - val_loss: 0.0138 - val_my_metric_fn: 0.0446
Epoch 3/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0111 - my_metric_fn: 0.0879 - val_loss: 0.0136 - val_my_metric_fn: 0.0325
Epoch 4/50
2813/2813 [==============================] - 4s 2ms/step - loss: 0.0139 - my_metric_fn: 0.0396
[0.013902746140956879, 0.03963005915284157]
2813/2813 [==============================] - 4s 1ms/step

Epoch 1/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0125 - my_metric_fn: -0.0391 - val_loss: 0.0145 - val_my_metric_fn: 0.0091
Epoch 2/50
2761/2761 [==============================] - 7s 3ms/step - loss: 0.0119 - my_metric_fn: 0.0226 - val_loss: 0.0139 - val_my_metric_fn: 0.0394
Epoch 3/50
2761/2761 [========

Epoch 1/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0125 - my_metric_fn: -0.0396 - val_loss: 0.0144 - val_my_metric_fn: 0.0100
Epoch 2/50
2761/2761 [==============================] - 7s 3ms/step - loss: 0.0118 - my_metric_fn: 0.0246 - val_loss: 0.0138 - val_my_metric_fn: 0.0462
Epoch 3/50
2761/2761 [==============================] - 7s 3ms/step - loss: 0.0110 - my_metric_fn: 0.0905 - val_loss: 0.0136 - val_my_metric_fn: 0.0319
Epoch 4/50
2813/2813 [==============================] - 5s 2ms/step - loss: 0.0139 - my_metric_fn: 0.0402
[0.013861685059964657, 0.04016318544745445]
2813/2813 [==============================] - 4s 2ms/step

Epoch 1/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0125 - my_metric_fn: -0.0382 - val_loss: 0.0144 - val_my_metric_fn: 0.0108
Epoch 2/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0118 - my_metric_fn: 0.0276 - val_loss: 0.0138 - val_my_metric_fn: 0.0437
Epoch 3/50
2761/2761 [========

Epoch 1/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0125 - my_metric_fn: -0.0385 - val_loss: 0.0144 - val_my_metric_fn: 0.0097
Epoch 2/50
2761/2761 [==============================] - 6s 2ms/step - loss: 0.0119 - my_metric_fn: 0.0231 - val_loss: 0.0139 - val_my_metric_fn: 0.0402
Epoch 3/50
2761/2761 [==============================] - 6s 2ms/step - loss: 0.0111 - my_metric_fn: 0.0856 - val_loss: 0.0136 - val_my_metric_fn: 0.0341
Epoch 4/50
2813/2813 [==============================] - 4s 2ms/step - loss: 0.0139 - my_metric_fn: 0.0406
[0.013891768641769886, 0.04062218219041824]
2813/2813 [==============================] - 4s 1ms/step

Epoch 1/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0125 - my_metric_fn: -0.0381 - val_loss: 0.0144 - val_my_metric_fn: 0.0100
Epoch 2/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0118 - my_metric_fn: 0.0261 - val_loss: 0.0138 - val_my_metric_fn: 0.0434
Epoch 3/50
2761/2761 [========

Epoch 1/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0125 - my_metric_fn: -0.0392 - val_loss: 0.0144 - val_my_metric_fn: 0.0106
Epoch 2/50
2761/2761 [==============================] - 6s 2ms/step - loss: 0.0118 - my_metric_fn: 0.0270 - val_loss: 0.0138 - val_my_metric_fn: 0.0417
Epoch 3/50
2761/2761 [==============================] - 6s 2ms/step - loss: 0.0111 - my_metric_fn: 0.0883 - val_loss: 0.0136 - val_my_metric_fn: 0.0313
Epoch 4/50
2813/2813 [==============================] - 5s 2ms/step - loss: 0.0139 - my_metric_fn: 0.0437
[0.013856420293450356, 0.04366641491651535]
2813/2813 [==============================] - 4s 1ms/step

0.12049795180438982


In [19]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den


def build(n_inputs=49,):
    l1_reg = 0.0
    l2_reg = 0.0
    dropout = 0.0
    lr = 0.0001
        
    # Define the hidden layers 
    input1 = Input(shape=(49,))
    x1 = Dense(6, activation='relu')(input1)
        
    # Input 2
    input2 = Input(shape=(49,))
    x2 = Dense(32, activation='relu',kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(input2)
    x2 = Dense(16, activation='relu',kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(x2)
    #x2 = Dense(8, activation='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(x2)
    x2 = Dense(6, activation='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(x2)
        
    merged = Multiply()([x1, x2])
    output = Dense(1, activation='linear')(merged)
 
    # Create the model 
    model = Model(inputs=[input1, input2], outputs=output)
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
    return model

In [20]:
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()
# 10 Trials for Prediction
merged_train_latent_all = np.concatenate((merged_train_latent, merged_val_latent))
merged_train_firm_all = np.concatenate((merged_train_firm, merged_val_firm))
merged_y_all = np.concatenate((merged_train_y, merged_val_y))

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    model = build()
    model.fit([merged_train_firm, merged_train_latent], merged_train_y,
              validation_data=([merged_val_firm, merged_val_latent], merged_val_y),
              epochs=30,
              batch_size = 128,
              callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)]
              )
    print(model.evaluate([merged_test_firm, merged_test_latent], merged_test_y))
    y_hat = model.predict([merged_test_firm, merged_test_latent]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

Epoch 1/30
691/691 [==============================] - 6s 6ms/step - loss: 0.0119 - my_metric_fn: 0.0239 - val_loss: 0.0142 - val_my_metric_fn: 0.0222
Epoch 2/30
691/691 [==============================] - 4s 6ms/step - loss: 0.0114 - my_metric_fn: 0.0602 - val_loss: 0.0141 - val_my_metric_fn: 0.0209
Epoch 3/30
691/691 [==============================] - 4s 5ms/step - loss: 0.0111 - my_metric_fn: 0.0834 - val_loss: 0.0141 - val_my_metric_fn: 0.0160
Epoch 4/30
2813/2813 [==============================] - 9s 3ms/step - loss: 0.0140 - my_metric_fn: 0.0374
[0.014004412107169628, 0.03736567124724388]
2813/2813 [==============================] - 9s 3ms/step

Epoch 1/30
691/691 [==============================] - 5s 6ms/step - loss: 0.0119 - my_metric_fn: 0.0205 - val_loss: 0.0142 - val_my_metric_fn: 0.0254
Epoch 2/30
691/691 [==============================] - 3s 4ms/step - loss: 0.0113 - my_metric_fn: 0.0763 - val_loss: 0.0140 - val_my_metric_fn: 0.0295
Epoch 3/30
691/691 [======================

Epoch 2/30
691/691 [==============================] - 2s 2ms/step - loss: 0.0111 - my_metric_fn: 0.0916 - val_loss: 0.0139 - val_my_metric_fn: 0.0274
Epoch 3/30
691/691 [==============================] - 2s 2ms/step - loss: 0.0105 - my_metric_fn: 0.1363 - val_loss: 0.0139 - val_my_metric_fn: 0.0255
Epoch 4/30
2813/2813 [==============================] - 4s 1ms/step - loss: 0.0140 - my_metric_fn: 0.0390
[0.01399341318756342, 0.038965798914432526]
2813/2813 [==============================] - 4s 1ms/step



In [21]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.16251541798093383


In [18]:
max_trials = 200
train_epochs = 200

tuner = keras_tuner.RandomSearch(MyHyperModel(),objective='val_loss',max_trials=max_trials,overwrite=True,)
tuner.search([merged_train_firm, merged_train_latent], merged_train_y, 
             epochs=train_epochs, 
             validation_data=([merged_val_firm, merged_val_latent], merged_val_y),
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

Trial 2 Complete [00h 00m 13s]
val_loss: 0.01379997469484806

Best val_loss So Far: 0.01379997469484806
Total elapsed time: 00h 00m 28s
INFO:tensorflow:Oracle triggered exit


In [7]:
# 10 Trials for Prediction
# 10 Trials for Prediction
merged_train_latent_all = np.concatenate((merged_train_latent, merged_val_latent))
merged_train_firm_all = np.concatenate((merged_train_firm, merged_val_firm))
merged_y_all = np.concatenate((merged_train_y, merged_val_y))
best_hps = tuner.get_best_hyperparameters(5)
print(best_hps[0].values)
print(best_hps[1].values)
print(best_hps[2].values)

{'l1_ratio': 0.0, 'l2_ratio': 0.0, 'dropout': 0.0, 'learning_rate': 0.0001, 'batch_size': 256}
{'l1_ratio': 0.0, 'l2_ratio': 0.0, 'dropout': 0.0, 'learning_rate': 0.0001, 'batch_size': 128}
{'l1_ratio': 0.0, 'l2_ratio': 0.0, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 256}


In [19]:
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = tuner.get_best_models(3)[0]
    best_model.fit([merged_train_latent, merged_train_firm], merged_train_y,
                   epochs=50,
                   validation_data=([merged_val_latent, merged_val_firm], merged_val_y),
                   callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
    print(best_model.evaluate([merged_test_latent, merged_test_firm], merged_test_y))
    y_hat = best_model.predict([merged_test_latent, merged_test_firm]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

Epoch 1/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0117 - my_metric_fn: 0.0266 - val_loss: 0.0138 - val_my_metric_fn: 0.0302
Epoch 2/50
2813/2813 [==============================] - 5s 2ms/step - loss: 0.0139 - my_metric_fn: 0.0475
[0.013854393735527992, 0.04747656360268593]
2813/2813 [==============================] - 4s 2ms/step

Epoch 1/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0117 - my_metric_fn: 0.0270 - val_loss: 0.0139 - val_my_metric_fn: 0.0267
Epoch 2/50
2813/2813 [==============================] - 8s 3ms/step - loss: 0.0139 - my_metric_fn: 0.0494
[0.013876578770577908, 0.04936505854129791]
2813/2813 [==============================] - 5s 2ms/step

Epoch 1/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0118 - my_metric_fn: 0.0247 - val_loss: 0.0138 - val_my_metric_fn: 0.0319
Epoch 2/50
2813/2813 [==============================] - 5s 2ms/step - loss: 0.0138 - my_metric_fn: 0.0477
[0.013845900073647499,